# Looks like segmentation is incorrect for Case7&Day0

I found that data of Case7&Day0 is incorrect. 
I wonder if i am using the incorrect method to decode the mask. 
If I do something wrong, please let me know. 

In [ ]:
import pandas as pd
import cv2
import random
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

In [ ]:
TRAIN_DIR="../input/uw-madison-gi-tract-image-segmentation/train"
file_list = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive=True)
file_list = pd.Series(file_list)

In [ ]:
def noseg(l):
    a = ""
    for i in l:
        a += i
    # no segmentation
    if a == "":
        return True
    else:
        return False

def imgpath(id):
    s = id.split('_')
    case = s[0]
    day = s[1]
    slc = s[3]
    path = file_list[file_list.str.contains(case + '_')&\
              file_list.str.contains('_' + day)&\
              file_list.str.contains('slice_' + slc)]
    assert(len(path) == 1)
    return path.iloc[0]

In [ ]:
df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
df = df.fillna("")
seg = df.groupby('id')['segmentation'].agg(list).to_frame().reset_index()
df = df.drop(columns=["class", 'segmentation'])
df = df.groupby('id').count()
df = df.merge(seg, on='id')
df['noseg'] = df.segmentation.apply(noseg)
df = df[df.noseg == False].reset_index()
df.head(5)

In [ ]:
# case: case number
# day: day
# num: number of sample
def showCase(case, day, num = 10):
    global df
    scase = f"case{case}_"
    sday = f"_day{day}_"
    print(scase, sday)
    tmpdf = df[df.id.str.contains(scase)&df.id.str.contains(sday)]
    id = random.sample(set(tmpdf.id.unique()), num)
    tmpdf = tmpdf[tmpdf.id.isin(id)]
    tmpdf['filepath'] = tmpdf.id.apply(imgpath)
#     print(tmpdf)
    fig, ax = plt.subplots(num, 3, figsize=(25, 8*num))
    for i in range(num):
        record = tmpdf.iloc[i]
        img = cv2.imread(record.filepath, cv2.IMREAD_ANYDEPTH)
        ax[i, 0].imshow(img)
        ax[i, 0].set_title(record.filepath)
        mask = np.zeros(img.shape)
        for n in range(3):
            mask += rle_decode(record.segmentation[n], img.shape)*(n+1)/4*np.max(img)
        ax[i, 1].imshow(mask)
        ax[i, 2].imshow(img + mask)

# Case7&Day0

In [ ]:
showCase(case=7, day=0, num=5)

# Case7&Day13

In [ ]:
showCase(case=7, day=13, num=5)

# Case7&Day19

In [ ]:
showCase(case=7, day=19, num=5)

# Random Case&Day

In [ ]:
ids = df.sample(10).id
for i in ids:
    s = i.split('_')
    case = s[0][4:]
    day = s[1][3:]
    showCase(case=case, day=day, num=5)
    